In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

feature_name_df = pd.read_csv('/content/features.txt',sep ='\s+', header=None, names=['column_index', 'column_name'])

FileNotFoundError: ignored

In [ ]:
feature_name = feature_name_df.iloc[:,1].values.tolist()

In [ ]:
# 중북 feature 제거
def get_new_feature(df):
  # cumcount = groupby에서 나눠진 것중에 index 매김
  feature_dup_df  = pd.DataFrame(data = df.groupby('column_name').cumcount(), columns = ['dup_cnt'])
  feature_dup_df = feature_dup_df.reset_index()
  new_df = pd.merge(df.reset_index(), feature_dup_df, how='outer')
  new_df['column_name'] = new_df[['column_name','dup_cnt']].apply(lambda x: x[0]+ '_'+ str(x[1]) if x[1]>0 else x[0], axis=1)
  new_df = new_df.drop(['index'], axis=1)
  return new_df

In [ ]:
def get_human():

  raw_feature = pd.read_csv('/content/features.txt',sep ='\s+',header=None,names=['column_index', 'column_name'])
  new_feature = get_new_feature(raw_feature)
  feature_name = new_feature.iloc[:,1].values.tolist()
  X_train = pd.read_csv('/content/X_train.txt',sep ='\s+',names=feature_name)
  X_test = pd.read_csv('/content/X_test.txt',sep ='\s+',names=feature_name)

  y_train = pd.read_csv('/content/y_train.txt',sep ='\s+', names=['action'])
  y_test  = pd.read_csv('/content/y_test.txt',sep ='\s+',names = ['action'])

  return X_train,y_train,y_test,X_test

In [ ]:
X_train,y_train,y_test,X_test = get_human()

In [ ]:
print(y_train['action'].value_counts())

6    1407
5    1374
4    1286
1    1226
2    1073
3     986
Name: action, dtype: int64


In [ ]:
# Train and run

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
dt_clf = DecisionTreeClassifier(random_state=111)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)

acu=accuracy_score(y_test, pred)
print(f'score: {acu}')

score: 0.8486596538853071


In [ ]:
print(dt_clf.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 111, 'splitter': 'best'}


## What is Ensemble?

In [ ]:
# voting Classfier

import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
cancer = load_breast_cancer()
data = pd.DataFrame(cancer.data, columns = cancer.feature_names)
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
lr = LogisticRegression(solver = 'liblinear')
knn = KNeighborsClassifier(n_neighbors = 8)

vo_clf = VotingClassifier( estimators = [('LR', lr), ('KNN', knn)], voting = 'soft')

X_train, X_test, y_train, y_test  = train_test_split(cancer.data, cancer.target, test_size = 0.2, random_state = 2012)

vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)

print(f'정확도 {accuracy_score(y_test, pred)}')

정확도 0.9473684210526315


In [ ]:
classfiers = [lr, knn]
for classfier in classfiers:
  classfier.fit(X_train, y_train)
  pred = classfier.predict(X_test)
  class_name = classfier.__class__.__name__
  print(f'{class_name} 정확도 : {accuracy_score(y_test, pred)}')

LogisticRegression 정확도 : 0.9649122807017544
KNeighborsClassifier 정확도 : 0.9298245614035088


In [ ]:
# How to use RandomForest
# max_features [default]: sqrt
# cpu 코어가 많으면 병렬 처리가 빨라 random Forest가 좋아진다
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth':[8,16,24],
    'min_samples_leaf':[1,6,12],
    'min_samples_split':[2,8,16]
}

rt_clf = RandomForestClassifier(n_estimators=100, random_state =0, n_jobs = -1)
grid_cv = GridSearchCV(rt_clf, param_grid = params, cv=2 , n_jobs = -1)
grid_cv.fit(X_train, y_train)


GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [8, 16, 24],
                         'min_samples_leaf': [1, 6, 12],
                         'min_samples_split': [2, 8, 16]})

In [ ]:
print(grid_cv.best_params_)

{'max_depth': 8, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [ ]:

hyper_rt_clf = RandomForestClassifier(n_estimators=100, max_depth= 8, min_samples_leaf= 1, min_samples_split= 2, random_state = 0)
hyper_rt_clf.fit(X_train, y_train)
pred = hyper_rt_clf.predict(X_test)
print(accuracy_score(y_test,pred))


0.9385964912280702


Look at Feature Importance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

feature_value = hyper_rt_clf.feature_importances_
print(feature_value)
feature = pd.Series(feature_value, index = X_train.columns)
top20 = feature.sort_values(ascending = False)[:20]

[0.0258983  0.01052335 0.06965222 0.04534171 0.00673925 0.00408725
 0.0656084  0.11492703 0.00428856 0.00257298 0.0235884  0.00498746
 0.01623337 0.03485034 0.00572442 0.00397679 0.00373342 0.00604953
 0.00413284 0.00549261 0.1147597  0.01284429 0.17449864 0.08491746
 0.01151002 0.00957903 0.01698034 0.09777808 0.01036261 0.00836164]


AttributeError: ignored

### What is GBM

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import warnings
import time
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

start_time = time.time()
gb_clf = GradientBoostingClassifier(random_state = 0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print(f'accuracy score: {gb_accuracy}')
print(f'time is {time.time()-start_time}')

NameError: ignored

In [ ]:

s